# Training TrackNet for Tennis Ball Tracking

## Introduction and Setup

This` notebook implements the TrackNet model for tennis ball tracking. TrackNet is a deep learning model that uses a sequence of three consecutive frames to predict the location of a tennis ball in the current frame. The model outputs a heatmap where the brightest region corresponds to the predicted ball location.

### Key Features:
- Uses a CNN with VGG16-style encoder and decoder architecture
- Takes 3 consecutive frames as input (9 channels total)
- Outputs a 256-class heatmap for ball localization
- Implements the exact training protocol from the original TrackNet paper

### Import Required Libraries

Import all necessary libraries for deep learning, data processing, and visualization.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tensorboardX import SummaryWriter
import numpy as np
import pandas as pd
import cv2
import os
from pathlib import Path
from tqdm import tqdm
import math

### Configuration and Setup

Define the configuration parameters for training TrackNet, including data paths, model hyperparameters, and training settings. These parameters are based on the original TrackNet paper specifications.

In [2]:
class Config:
    #Data Paths
    RAW_DATASET_DIR = Path('data/tracknet_dataset')
    PREPROCESSED_DATASET_DIR = Path('data/preprocessed_tracknet_dataset')

    # Model & Training Hyperparameters (from the paper)
    INPUT_WIDTH = 640
    INPUT_HEIGHT = 360
    BATCH_SIZE = 2
    NUM_EPOCHS = 500
    LEARNING_RATE = 1.0
    VAL_INTERVALS = 5
    STEPS_PER_EPOCH = 200 # As specified in the paper
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    

config = Config()
PREPROCESSED_IMAGES_DIR = config.PREPROCESSED_DATASET_DIR / 'images'
PREPROCESSED_GTS_DIR = config.PREPROCESSED_DATASET_DIR / 'gts'
MODEL_PATH = Path(f'./models')
TB_PATH = MODEL_PATH / 'plots'
MODEL_BEST_PATH = MODEL_PATH / 'model_best.pt'
MODEL_LAST_PATH = MODEL_PATH / 'model_last.pt'

os.makedirs(config.PREPROCESSED_DATASET_DIR, exist_ok=True)
os.makedirs(PREPROCESSED_IMAGES_DIR, exist_ok=True)
os.makedirs(PREPROCESSED_GTS_DIR, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)
os.makedirs(TB_PATH, exist_ok=True)

print(f"Using device: {config.DEVICE}")
print(f"Raw data path: {config.RAW_DATASET_DIR}")
print(f"Prepared data will be saved to: {config.PREPROCESSED_DATASET_DIR}")

Using device: cuda
Raw data path: data\tracknet_dataset
Prepared data will be saved to: data\preprocessed_tracknet_dataset


## Step 1: Data Preparation (Ground Truth Generation)

This section prepares the training data by:
1. **Creating Ground Truth Heatmaps**: Converts ball coordinates to Gaussian heatmaps for training
2. **Organizing Label Files**: Creates train/validation splits and aligns 3-frame sequences
3. **Copying Images**: Reorganizes raw images into a unified directory structure

The ground truth generation follows the TrackNet paper methodology, creating 256-intensity heatmaps with Gaussian kernels centered at ball locations.

In [ ]:
def create_gaussian(size, variance):
    """
    Create a 2D Gaussian kernel for ground truth heatmap generation.
    
    Args:
        size (int): Half-size of the kernel (full size = 2*size + 1)
        variance (float): Gaussian variance parameter
        
    Returns:
        np.ndarray: 2D Gaussian kernel with values 0-255
    """
    x, y = np.mgrid[-size:size+1, -size:size+1]
    g = np.exp(-(x**2 + y**2) / float(2 * variance))
    g = g * 255 / g.max()  # Normalize to 0-255 range
    return g.astype(np.uint8)

def create_gt_images(path_input, path_output, size, variance, width, height):
    """
    Generate ground truth heatmap images for tennis ball training data.
    
    Creates Gaussian heatmaps centered at ball coordinates for each frame.
    Invisible balls (visibility=0) or NaN coordinates result in zero heatmaps.
    
    Args:
        path_input (Path): Input directory containing game/clip subdirectories
        path_output (Path): Output directory for generated heatmaps
        size (int): Gaussian kernel half-size
        variance (float): Gaussian variance
        width (int): Image width (1280 for original resolution)
        height (int): Image height (720 for original resolution)
    """
    print("--- Starting Ground Truth Image Generation ---")
    gaussian_kernel_array = create_gaussian(size, variance)
    
    for game_dir in tqdm(sorted(path_input.glob("game*")), desc="Processing Games"):
        for clip_dir in sorted(game_dir.glob("Clip*")):
            path_out_clip = path_output / game_dir.name / clip_dir.name
            os.makedirs(path_out_clip, exist_ok=True)
            
            labels_path = clip_dir / 'Label.csv'
            if not labels_path.exists():
                continue
            
            labels = pd.read_csv(labels_path)
            for _, row in labels.iterrows():
                file_name, vis, x, y = row['file name'], row['visibility'], row['x-coordinate'], row['y-coordinate']
                heatmap = np.zeros((height, width, 3), dtype=np.uint8)
                
                # Only create heatmap for visible balls with valid coordinates
                if vis != 0 and not (math.isnan(x) or math.isnan(y)):
                    x, y = int(x), int(y)
                    
                    # Place Gaussian kernel at ball position
                    for i in range(-size, size + 1):
                        for j in range(-size, size + 1):
                            if 0 <= y + j < height and 0 <= x + i < width:
                                temp = gaussian_kernel_array[j + size, i + size]
                                if temp > 0:
                                    heatmap[y + j, x + i] = (temp, temp, temp)
                
                cv2.imwrite(str(path_out_clip / file_name), heatmap)
    print("--- Ground Truth Image Generation Complete ---")


def create_gt_labels(path_input, path_gts, path_output, train_rate=0.7):
    """
    Create training and validation CSV files with 3-frame sequences.
    
    Processes all game clips to create aligned frame sequences (t-2, t-1, t)
    and corresponding ground truth paths. Randomly splits into train/val sets.
    
    Args:
        path_input (Path): Raw dataset directory
        path_gts (Path): Ground truth heatmaps directory (unused in function)
        path_output (Path): Output directory for CSV files
        train_rate (float): Training split ratio (default: 0.7)
    """
    print("--- Starting Label File Generation ---")
    df = pd.DataFrame()
    
    for game_dir in tqdm(sorted(path_input.glob("game*")), desc="Processing Labels"):
        for clip_dir in sorted(game_dir.glob("Clip*")):
            labels_path = clip_dir / 'Label.csv'
            if not labels_path.exists():
                continue
            
            labels = pd.read_csv(labels_path)
            
            game_name = game_dir.name
            clip_name = clip_dir.name
            
            # Create unified path structure for preprocessed data
            labels['gt_path'] = labels['file name'].apply(lambda fn: f'gts/{game_name}/{clip_name}/{fn}')
            labels['path1'] = labels['file name'].apply(lambda fn: f'images/{game_name}/{clip_name}/{fn}')
            
            # Create 3-frame sequences: [t-2, t-1, t] where t is the target frame
            labels_target = labels[2:].copy().reset_index(drop=True)  # Target frame (t)
            labels_target['path2'] = labels['path1'][1:-1].reset_index(drop=True)  # t-1
            labels_target['path3'] = labels['path1'][:-2].reset_index(drop=True)   # t-2
            
            df = pd.concat([df, labels_target])

    # Prepare final dataset
    df = df.reset_index(drop=True)
    df = df[['path1', 'path2', 'path3', 'gt_path', 'x-coordinate', 'y-coordinate', 'status', 'visibility']]
    df = df.sample(frac=1).reset_index(drop=True)  # Shuffle data
    
    # Split into train/validation sets
    num_train = int(df.shape[0] * train_rate)
    df_train = df.iloc[:num_train]
    df_val = df.iloc[num_train:]
    
    df_train.to_csv(path_output / 'labels_train.csv', index=False)
    df_val.to_csv(path_output / 'labels_val.csv', index=False)
    print(f"Training labels saved ({len(df_train)} samples)")
    print(f"Validation labels saved ({len(df_val)} samples)")
    print("--- Label File Generation Complete ---")

# Execute data preparation pipeline
if not (config.PREPROCESSED_DATASET_DIR / 'labels_train.csv').exists():
    # Generate ground truth heatmaps at original resolution
    create_gt_images(
        path_input=config.RAW_DATASET_DIR,
        path_output=PREPROCESSED_GTS_DIR,
        size=20, variance=10, width=1280, height=720
    )
    
    # Copy raw images to unified directory structure
    print("Copying original images to prepared directory...")
    for game_dir in tqdm(sorted(config.RAW_DATASET_DIR.glob("game*"))):
        for clip_dir in sorted(game_dir.glob("Clip*")):
            dest_clip_dir = PREPROCESSED_IMAGES_DIR / game_dir.name / clip_dir.name
            os.makedirs(dest_clip_dir, exist_ok=True)
            
            for img_file in clip_dir.glob("*.jpg"):
                os.system(f'cp "{img_file}" "{dest_clip_dir / img_file.name}"')
    
    # Generate train/validation CSV files
    create_gt_labels(
        path_input=config.RAW_DATASET_DIR,
        path_gts=PREPROCESSED_GTS_DIR,
        path_output=config.PREPROCESSED_DATASET_DIR
    )
else:
    print("Prepared dataset files already exist. Skipping generation.")

Prepared dataset files already exist. Skipping generation.


## Step 2: PyTorch Dataset Class

The `TrackNetDataset` class handles loading and preprocessing of the training data:

- **Input**: 3 consecutive frames (concatenated as 9-channel tensor)
- **Output**: Ground truth heatmap with ball location 
- **Preprocessing**: Resizes images to 640x360, normalizes pixel values, and scales coordinates
- **Data Loading**: Reads from preprocessed CSV files and image directories

This dataset implementation ensures proper temporal alignment and handles cases where the ball is not visible.

In [4]:
class TrackNetDataset(Dataset):
    def __init__(self, mode, input_height=360, input_width=640):
        self.path_dataset = config.PREPROCESSED_DATASET_DIR
        assert mode in ['train', 'val'], 'incorrect mode'
        self.data = pd.read_csv(os.path.join(self.path_dataset, f'labels_{mode}.csv'))
        print(f'Mode = {mode}, Samples = {self.data.shape[0]}')
        self.height = input_height
        self.width = input_width
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        path, path_prev, path_preprev = row['path1'], row['path2'], row['path3']
        path_gt = row['gt_path']
        x, y, vis = row['x-coordinate'], row['y-coordinate'], row['visibility']
        
        # Handle cases where there is no ball
        if vis == 0 or math.isnan(x):
            x, y = -1, -1

        # Rescale coordinates for the smaller input size
        x_scaled = x * (self.width / 1280)
        y_scaled = y * (self.height / 720)

        inputs = self._get_input(path, path_prev, path_preprev)
        outputs = self._get_output(path_gt)
        
        return inputs, outputs, x_scaled, y_scaled, vis

    def _get_output(self, path_gt):
        # The GT images were generated at 1280x720, so we resize them
        img_path = os.path.join(self.path_dataset, path_gt)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (self.width, self.height))
        # The loss function expects a 1D target for each pixel in the batch
        return torch.from_numpy(img).long()
        
    def _get_input(self, path, path_prev, path_preprev):
        imgs_list = []
        for p in [path_preprev, path_prev, path]:
            img_path = os.path.join(self.path_dataset, p)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (self.width, self.height))
            # BGR to RGB, normalize, and permute to (C, H, W)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_tensor = torch.from_numpy(img).float() / 255.0
            imgs_list.append(img_tensor.permute(2, 0, 1))
        
        return torch.cat(imgs_list, dim=0)

## Step 3: TrackNet Model Architecture

Implementation of the TrackNet neural network with:

### Encoder (VGG16-style):
- Multiple convolutional blocks with ReLU activation and batch normalization
- Progressive downsampling with max pooling (640x360 → 80x45)
- Feature extraction from 9 input channels (3 frames × 3 RGB channels)

### Decoder (DeconvNet-style):
- Upsampling layers to restore original resolution
- Outputs 256-class heatmap for ball localization
- Each class represents a different intensity level in the heatmap

The model architecture follows the exact specifications from the TrackNet paper.

In [ ]:
class ConvBlock(nn.Module):
    """
    Basic convolutional block with Conv2D + ReLU + BatchNorm.
    
    Args:
        in_channels (int): Number of input channels
        out_channels (int): Number of output channels
        kernel_size (int): Convolution kernel size (default: 3)
        pad (int): Padding size (default: 1)
        stride (int): Convolution stride (default: 1)
        bias (bool): Whether to use bias in convolution (default: True)
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, pad=1, stride=1, bias=True):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=pad, bias=bias),
            nn.ReLU(),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.block(x)

class TrackNet(nn.Module):
    """
    TrackNet model for tennis ball tracking using encoder-decoder architecture.
    
    Takes 3 consecutive frames (9 channels) as input and outputs a 256-class
    heatmap for ball localization. Architecture follows VGG16-style encoder
    with DeconvNet-style decoder for pixel-level classification.
    
    Args:
        out_channels (int): Number of output classes (default: 256)
    """
    def __init__(self, out_channels=256):
        super().__init__()
        self.out_channels = out_channels

        # Encoder: VGG16-style feature extraction
        # Block 1: 9 -> 64 channels, spatial size: 640x360 -> 320x180
        self.conv1 = ConvBlock(in_channels=9, out_channels=64)
        self.conv2 = ConvBlock(in_channels=64, out_channels=64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Block 2: 64 -> 128 channels, spatial size: 320x180 -> 160x90
        self.conv3 = ConvBlock(in_channels=64, out_channels=128)
        self.conv4 = ConvBlock(in_channels=128, out_channels=128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Block 3: 128 -> 256 channels, spatial size: 160x90 -> 80x45
        self.conv5 = ConvBlock(in_channels=128, out_channels=256)
        self.conv6 = ConvBlock(in_channels=256, out_channels=256)
        self.conv7 = ConvBlock(in_channels=256, out_channels=256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Block 4: 256 -> 512 channels (bottleneck), spatial size: 80x45
        self.conv8 = ConvBlock(in_channels=256, out_channels=512)
        self.conv9 = ConvBlock(in_channels=512, out_channels=512)
        self.conv10 = ConvBlock(in_channels=512, out_channels=512)

        # Decoder: DeconvNet-style upsampling
        # Upsample 1: 80x45 -> 160x90, 512 -> 256 channels
        self.ups1 = nn.Upsample(scale_factor=2)
        self.conv11 = ConvBlock(in_channels=512, out_channels=256)
        self.conv12 = ConvBlock(in_channels=256, out_channels=256)
        self.conv13 = ConvBlock(in_channels=256, out_channels=256)

        # Upsample 2: 160x90 -> 320x180, 256 -> 128 channels
        self.ups2 = nn.Upsample(scale_factor=2)
        self.conv14 = ConvBlock(in_channels=256, out_channels=128)
        self.conv15 = ConvBlock(in_channels=128, out_channels=128)

        # Upsample 3: 320x180 -> 640x360, 128 -> 64 -> out_channels
        self.ups3 = nn.Upsample(scale_factor=2)
        self.conv16 = ConvBlock(in_channels=128, out_channels=64)
        self.conv17 = ConvBlock(in_channels=64, out_channels=64)
        self.conv18 = ConvBlock(in_channels=64, out_channels=self.out_channels)

        self.softmax = nn.Softmax(dim=1)
        self._init_weights()
                  
    def forward(self, x):
        """
        Forward pass through TrackNet.
        
        Args:
            x (torch.Tensor): Input tensor of shape (N, 9, H, W)
            
        Returns:
            torch.Tensor: Output heatmap of shape (N, 256, H, W)
        """
        # Encoder path
        x = self.conv1(x)
        x = self.conv2(x)    
        x = self.pool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        
        # Decoder path
        x = self.ups1(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.ups2(x)
        x = self.conv14(x)
        x = self.conv15(x)
        x = self.ups3(x)
        x = self.conv16(x)
        x = self.conv17(x)
        x = self.conv18(x)

        return x                       
    
    def _init_weights(self):
        """Initialize model weights following TrackNet paper specifications."""
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.uniform_(module.weight, -0.05, 0.05)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)


# Test model instantiation and forward pass
model = TrackNet().to(config.DEVICE)
inp = torch.randn(config.BATCH_SIZE, 9, config.INPUT_HEIGHT, config.INPUT_WIDTH).to(config.DEVICE)
out = model(inp)
print('out = {}'.format(out.shape))

out = torch.Size([2, 256, 360, 640])


## Step 4: Training and Validation Logic

This section implements the core training and evaluation functions:

### Helper Functions:
- **`get_coords_from_heatmap()`**: Extracts ball coordinates from predicted heatmaps
- **`calculate_metrics()`**: Computes precision, recall, and F1 score
- **`postprocess()`**: Applies Hough Circle Transform to refine predictions

### Validation Strategy:
- Granular metrics per visibility class (easy, hard, occluded)
- Distance-based accuracy measurement (5-pixel threshold)
- Performance tracking across different ball visibility conditions

### Utility Functions for Evaluation

Helper functions to extract ball coordinates from heatmaps and calculate performance metrics.

In [6]:
def get_coords_from_heatmap(heatmap):
    """
    Finds the (x, y) coordinates of the maximum value in a heatmap.
    """
    max_val = heatmap.max()
    if max_val == 0:
        return -1, -1 # No prediction
        
    coords = (heatmap == max_val).nonzero()
    # Take the first one if there are multiple max values
    return coords[0, 1].item(), coords[0, 0].item()

def calculate_metrics(pred_coords, gt_coords, threshold=5):
    """
    Calculates TP, FP, FN based on distance between predicted and GT coordinates.
    """
    tp, fp, fn = 0, 0, 0
    
    for (px, py), (gx, gy) in zip(pred_coords, gt_coords):
        gt_ball_present = (gx != -1)
        pred_ball_present = (px != -1)
        
        if gt_ball_present and pred_ball_present:
            dist = math.sqrt((px - gx)**2 + (py - gy)**2)
            if dist <= threshold:
                tp += 1
            else:
                fp += 1 # Prediction was too far away
                fn += 1 # Missed the actual ball
        elif gt_ball_present and not pred_ball_present:
            fn += 1 # Model failed to predict a ball that was there
        elif not gt_ball_present and pred_ball_present:
            fp += 1 # Model predicted a ball that wasn't there
            
    return tp, fp, fn

### Training and Validation Functions

Core functions for:
- **Training**: Forward/backward pass, loss calculation, and parameter updates
- **Postprocessing**: Hough Circle Transform for refined ball detection
- **Validation**: Comprehensive evaluation with per-class metrics and distance-based accuracy

In [ ]:
import time
from scipy.spatial import distance

def train(model, loader, optimizer, loss_fn, device, epoch, steps_per_epoch):
    """
    Execute one training epoch with specified number of steps.
    
    Args:
        model (nn.Module): TrackNet model to train
        loader (DataLoader): Training data loader
        optimizer (torch.optim): Optimizer for parameter updates
        loss_fn (nn.Module): Loss function (CrossEntropyLoss)
        device (str): Device to run training on ('cuda' or 'cpu')
        epoch (int): Current epoch number
        steps_per_epoch (int): Maximum steps per epoch (200 as per paper)
        
    Returns:
        float: Average loss for the epoch
    """
    model.train()
    total_loss = 0
    
    # Progress bar for in-training updates
    pbar = tqdm(loader, total=steps_per_epoch, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [Train]")
    
    for i, (inputs, targets, _, _, _) in enumerate(pbar):
        if i >= steps_per_epoch:
            break
            
        inputs = inputs.to(device)
        targets = targets.to(device)  # Shape: (N, H, W)
        
        # Forward pass: predict 256-class heatmap
        predictions = model(inputs)  # Shape: (N, 256, H, W)
        
        # Compute pixel-wise classification loss
        loss = loss_fn(predictions, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        pbar.set_postfix(loss=f'{loss.item():.4f}')
        
    return total_loss / steps_per_epoch

def postprocess(heatmap):
    """
    Extract ball coordinates from predicted heatmap using Hough Circle Transform.
    
    Applies binary thresholding followed by circle detection to find the most
    likely ball position. This follows the methodology from the TrackNet paper.
    
    Args:
        heatmap (np.ndarray): 2D heatmap with intensity values 0-255
        
    Returns:
        tuple: (x, y) coordinates of detected ball, or (None, None) if not found
    """
    heatmap = heatmap.astype(np.uint8)
    
    # Create binary mask from heatmap
    ret, binary_heatmap = cv2.threshold(heatmap, 127, 255, cv2.THRESH_BINARY)
    
    # Detect circular patterns in binary image
    circles = cv2.HoughCircles(binary_heatmap, cv2.HOUGH_GRADIENT, dp=1, minDist=1,
                               param1=50, param2=2, minRadius=2, maxRadius=7)
    
    x, y = None, None
    if circles is not None and len(circles) == 1:
        # Use center of detected circle as ball position
        x = circles[0][0][0]
        y = circles[0][0][1]
        
    return x, y

def validate(model, loader, loss_fn, device, epoch, min_dist_px=5):
    """
    Validate model performance with granular metrics per visibility class.
    
    Evaluates model predictions against ground truth using distance-based
    accuracy measurement. Tracks performance across different ball visibility
    conditions (easy, hard, occluded) as specified in TrackNet paper.
    
    Args:
        model (nn.Module): TrackNet model to validate
        loader (DataLoader): Validation data loader
        loss_fn (nn.Module): Loss function for validation loss
        device (str): Device to run validation on
        epoch (int): Current epoch number
        min_dist_px (int): Distance threshold for correct prediction (default: 5)
        
    Returns:
        tuple: (avg_loss, precision, recall, f1) validation metrics
    """
    model.eval()
    total_loss = 0
    
    # Initialize metric counters for visibility classes (0=none, 1=easy, 2=hard, 3=occluded)
    tp = [0] * 4  # True positives
    fp = [0] * 4  # False positives
    fn = [0] * 4  # False negatives
    tn = [0] * 4  # True negatives (only for vis=0)

    pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{config.NUM_EPOCHS} [Val]")

    with torch.no_grad():
        for inputs, targets, gt_x, gt_y, gt_vis in pbar:
            inputs = inputs.to(device)
            targets = targets.to(device)

            predictions = model(inputs)
            loss = loss_fn(predictions, targets)
            total_loss += loss.item()

            # Convert model output to predicted heatmaps
            pred_heatmaps = torch.argmax(predictions, dim=1).cpu().numpy()

            # Evaluate each sample in the batch
            for i in range(len(inputs)):
                px, py = postprocess(pred_heatmaps[i])
                gx, gy, gv = gt_x[i].item(), gt_y[i].item(), gt_vis[i].item()

                is_pred_ball = (px is not None)
                is_gt_ball = (gv != 0)

                if is_gt_ball:
                    if is_pred_ball:
                        # Check if prediction is within distance threshold
                        dist = distance.euclidean((px, py), (gx, gy))
                        if dist < min_dist_px:
                            tp[gv] += 1
                        else:
                            fp[gv] += 1  # Prediction too far from ground truth
                    else:
                        fn[gv] += 1  # Failed to detect existing ball
                else:  # No ground truth ball
                    if is_pred_ball:
                        fp[gv] += 1  # False detection
                    else:
                        tn[gv] += 1  # Correct no-ball prediction
                        
    avg_loss = total_loss / len(loader)
    
    # Calculate aggregate metrics for frames with visible balls (vis=1,2,3)
    total_tp = sum(tp[1:])
    total_fp = sum(fp[1:])
    total_fn = sum(fn[1:])
    
    eps = 1e-6
    precision = total_tp / (total_tp + total_fp + eps)
    recall = total_tp / (total_tp + total_fn + eps)
    f1 = 2 * (precision * recall) / (precision + recall + eps)

    # Print detailed breakdown by visibility class
    print(f"\n--- Validation Metrics (Epoch {epoch+1}) ---")
    print(f"  - Vis Class 1 (Easy): TP={tp[1]}, FP={fp[1]}, FN={fn[1]}")
    print(f"  - Vis Class 2 (Hard): TP={tp[2]}, FP={fp[2]}, FN={fn[2]}")
    print(f"  - Vis Class 3 (Occluded): TP={tp[3]}, FP={fp[3]}, FN={fn[3]}")
    print(f"  - No Ball (Vis 0): TN={tn[0]}, FP={fp[0]}")
    print("  ------------------------------------")
    
    return avg_loss, precision, recall, f1

## Step 5: Main Training Loop

Execute the complete training pipeline:

### Training Setup:
- Initialize datasets and data loaders for train/validation splits
- Configure the TrackNet model with Adadelta optimizer
- Set up TensorBoard logging for monitoring training progress

### Training Process:
- Train for 500 epochs with 200 steps per epoch (as per paper)
- Validate every 5 epochs with detailed performance metrics
- Save best model based on F1 score and latest model checkpoint
- Log all metrics to TensorBoard for visualization

This follows the exact training protocol specified in the TrackNet paper.

In [ ]:
print("--- Initializing Datasets and Dataloaders ---")
# Initialize the training dataset and dataloader
train_dataset = TrackNetDataset('train', input_height=config.INPUT_HEIGHT, input_width=config.INPUT_WIDTH)
train_loader = DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=True,
    num_workers=0, 
    pin_memory=True 
)

# Initialize the validation dataset and dataloader
val_dataset = TrackNetDataset('val', input_height=config.INPUT_HEIGHT, input_width=config.INPUT_WIDTH)
val_loader = DataLoader(
    val_dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

print("\n--- Initializing Model, Optimizer, and Loss ---")
# Instantiate the correct model architecture
model = TrackNet().to(config.DEVICE)

# The paper uses the Adadelta optimizer
optimizer = optim.Adadelta(model.parameters(), lr=config.LEARNING_RATE)

# The loss function for per-pixel classification is CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# Initialize TensorBoard for logging
log_writer = SummaryWriter(log_dir=str(TB_PATH))

# Variable to track the best validation metric (F1 score)
val_best_metric = 0.0

print("\n--- Starting Training ---")
# The main training loop
for epoch in range(config.NUM_EPOCHS):
    # Perform one epoch of training
    train_loss = train(model, train_loader, optimizer, loss_fn, config.DEVICE, epoch, config.STEPS_PER_EPOCH)
    
    # Print and log training results
    print(f"[Epoch {epoch+1}] Train Loss: {train_loss:.4f}")
    log_writer.add_scalar('Train/loss', train_loss, epoch)
    log_writer.add_scalar('Train/lr', optimizer.param_groups[0]['lr'], epoch)

    # Save the state of the model after the latest epoch
    torch.save(model.state_dict(), MODEL_LAST_PATH)

    # Run validation at specified intervals
    if (epoch + 1) % config.VAL_INTERVALS == 0:
        val_loss, precision, recall, f1 = validate(model, val_loader, loss_fn, config.DEVICE, epoch)
        
        # Print and log validation results
        print(f"[Epoch {epoch+1}] Val Loss: {val_loss:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        log_writer.add_scalar('Val/loss', val_loss, epoch)
        log_writer.add_scalar('Val/precision', precision, epoch)
        log_writer.add_scalar('Val/recall', recall, epoch)
        log_writer.add_scalar('Val/f1', f1, epoch)
        
        # If the current F1 score is the best we've seen, save the model state
        if f1 > val_best_metric:
            print(f"  -> New best F1 score: {f1:.4f}. Saving best model to {MODEL_BEST_PATH}")
            val_best_metric = f1
            torch.save(model.state_dict(), MODEL_BEST_PATH)

print("\n--- Training Complete ---")
print(f"Best validation F1 score: {val_best_metric:.4f}")
log_writer.close()

--- Initializing Datasets and Dataloaders ---
Mode = train, Samples = 13751
Mode = val, Samples = 5894

--- Initializing Model, Optimizer, and Loss ---

--- Starting Training ---


Epoch 1/500 [Train]: 100%|██████████| 200/200 [00:37<00:00,  5.34it/s, loss=4.0330]


[Epoch 1] Train Loss: 4.8701


Epoch 2/500 [Train]: 100%|██████████| 200/200 [00:37<00:00,  5.41it/s, loss=2.1814]


[Epoch 2] Train Loss: 3.1011


...

[Epoch 499] Train Loss: 0.0011


Epoch 500/500 [Train]: 100%|██████████| 200/200 [00:36<00:00,  5.47it/s, loss=0.0012]


[Epoch 500] Train Loss: 0.0011


Epoch 500/500 [Val]: 100%|██████████| 2947/2947 [04:06<00:00, 11.97it/s]


--- Validation Metrics (Epoch 500) ---
  - Vis Class 1 (Easy): TP=4347, FP=502, FN=409
  - Vis Class 2 (Hard): TP=248, FP=41, FN=134
  - Vis Class 3 (Occluded): TP=1, FP=1, FN=22
  - No Ball (Vis 0): TN=186, FP=3
  ------------------------------------
[Epoch 500] Val Loss: 0.0016, Precision: 0.8942, Recall: 0.8905, F1: 0.8923

--- Training Complete ---
Best validation F1 score: 0.9173


## Training Performance Visualization

Monitor training progress and model performance using TensorBoard. The following cells load the TensorBoard extension and display interactive plots showing:

- Training and validation loss curves
- Precision, recall, and F1 score metrics
- Learning rate schedule
- Model performance across different ball visibility classes

### Load TensorBoard Extension

Enable TensorBoard visualization within the Jupyter notebook environment.

In [4]:
%load_ext tensorboard

### Display TensorBoard Dashboard

Launch the TensorBoard interface to visualize training metrics and model performance in real-time.

In [6]:
%tensorboard --logdir models/plots

Reusing TensorBoard on port 6006 (pid 55428), started 4 days, 12:32:09 ago. (Use '!kill 55428' to kill it.)